In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
from scipy.spatial import distance

# Load YOLOv8 face detection model
model = YOLO('yolov8n-face.pt')

# Open video file
video_path = 'test_video.mp4'  # Input video file path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define VideoWriter to save output video
output_path = 'output_2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Define minimum distance to consider two faces the same
min_distance = 50  # Adjust based on video resolution and expected movement

# List to store unique face centers
unique_faces = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame with YOLOv8 face detection
    results = model(frame)

    # List to store the centers of detected faces in this frame
    current_faces = []

    for r in results:
        boxes = r.boxes  # Extract bounding boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            conf = box.conf  # Confidence score
            if conf >= 0.25:  # Confidence threshold for face detection
                # Calculate the center of the bounding box
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                current_faces.append((center_x, center_y))

                # Draw bounding box around the face
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Compare detected faces in this frame with unique faces tracked
    for face in current_faces:
        distances = [distance.euclidean(face, u_face) for u_face in unique_faces]
        if len(distances) == 0 or min(distances) > min_distance:
            # If no close match is found, consider it a new unique face
            unique_faces.append(face)

    # Write the frame with detections to the output video
    out.write(frame)

    # Display the frame (optional)
    cv2.imshow('YOLOv8 Face Detection', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer, close windows
cap.release()
out.release()  # Save the output video
cv2.destroyAllWindows()

# Output the total number of unique faces detected
print(f"Number of unique faces detected: {len(unique_faces)}")


c:\Users\hp\Desktop\yolo\yolov8-face\ultralytics\nn\tasks.py:362: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(weight, map_location='cpu')  # load
Ultraly

Number of unique faces detected: 127
